In [9]:

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
#TransferLearning of ATTACK --> 
# VS From Scratch
import torch
import torchvision
import torchvision.transforms as transforms
from utils import *
import torch.optim as optim
from model.baseline_model import *
from trainer import *
from torch.utils.tensorboard import SummaryWriter
import robustbench
#from robustbench.data import load_cifar10
from robustbench.utils import load_model, clean_accuracy
from finetune_framework import *

# Transfer Learning 

In [11]:
model_name = "resnet"
num_classes = 10
feature_extract = True

model_transfer, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
train_dataset, val_dataset, train_loader, val_loader = CIFAR10(128, finetune = True, 
                                                               input_size = input_size, test_batch_size=128)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
device = "cuda"


In [14]:
model_transfer = model_transfer.to(device)
params_to_update = model_transfer.parameters()

print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_transfer.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_transfer.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_transfer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 fc.weight
	 fc.bias


In [15]:

train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()
PATH = './cifar_resnet_transfer.pth'
device = "cuda"
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("Standard_Resnet_Transfer")


In [16]:
train(model_transfer, train_loader, train_criterion, val_loader, val_criterion, 
              optimizer_transfer, 100, writer, 
                 PATH, False, None, device="cuda",
                  n_steps_show=1000)

Epoch [0/100]	
Epoch [1/100]	 Training Loss: 1.622923265332761	 lr: 0.001
Epoch [1/100]	 Validation Loss: 1.016049538232103	 lr: 0.001
Epoch [1/100]	
Epoch [2/100]	 Training Loss: 1.2821733296069953	 lr: 0.001
Epoch [2/100]	 Validation Loss: 0.8903538313093065	 lr: 0.001
Epoch [2/100]	
Epoch [3/100]	 Training Loss: 1.2087412950632823	 lr: 0.001
Epoch [3/100]	 Validation Loss: 0.8157536327084408	 lr: 0.001
Epoch [3/100]	
Epoch [4/100]	 Training Loss: 1.1865175960923704	 lr: 0.001
Epoch [4/100]	 Validation Loss: 0.7940273511258862	 lr: 0.001
Epoch [4/100]	
Epoch [5/100]	 Training Loss: 1.160145839157007	 lr: 0.001
Epoch [5/100]	 Validation Loss: 0.7766734731348255	 lr: 0.001
Epoch [5/100]	
Epoch [6/100]	 Training Loss: 1.1493591211945808	 lr: 0.001
Epoch [6/100]	 Validation Loss: 0.7457914835290064	 lr: 0.001
Epoch [6/100]	
Epoch [7/100]	 Training Loss: 1.1355881354083186	 lr: 0.001
Epoch [7/100]	 Validation Loss: 0.7560576457011549	 lr: 0.001
Epoch [7/100]	
Epoch [8/100]	 Training Loss:

Epoch [60/100]	 Training Loss: 1.0713834283906785	 lr: 0.001
Epoch [60/100]	 Validation Loss: 0.670941551840758	 lr: 0.001
Epoch [60/100]	
Epoch [61/100]	 Training Loss: 1.0721073746681213	 lr: 0.001
Epoch [61/100]	 Validation Loss: 0.6775827098496353	 lr: 0.001
Epoch [61/100]	
Epoch [62/100]	 Training Loss: 1.0693825722655372	 lr: 0.001
Epoch [62/100]	 Validation Loss: 0.6710811781732342	 lr: 0.001
Epoch [62/100]	
Epoch [63/100]	 Training Loss: 1.0652119034086651	 lr: 0.001
Epoch [63/100]	 Validation Loss: 0.6665111863160436	 lr: 0.001
Epoch [63/100]	
Epoch [64/100]	 Training Loss: 1.072257290563315	 lr: 0.001
Epoch [64/100]	 Validation Loss: 0.6737492978572845	 lr: 0.001
Epoch [64/100]	
Epoch [65/100]	 Training Loss: 1.069849419776741	 lr: 0.001
Epoch [65/100]	 Validation Loss: 0.6781094926822034	 lr: 0.001
Epoch [65/100]	
Epoch [66/100]	 Training Loss: 1.0693810559294719	 lr: 0.001
Epoch [66/100]	 Validation Loss: 0.6657425632959679	 lr: 0.001
Epoch [66/100]	
Epoch [67/100]	 Trainin

In [17]:
print("the clean accuracy is ")
test(val_loader, model_transfer)


the clean accuracy is 
Accuracy of the network on the 10000 test images: 78 %


In [62]:
# pgd_attack_transfer = generate_adv(model_transfer_load, "pgd")
# fgsm_attack_transfer = generate_adv(model_transfer_load, "fgsm")
# nifgsm_attack_transfer = generate_adv(model_transfer_load, "nifgsm")
# vmifgsm_attack_transfer = generate_adv(model_transfer_load, "vmifgsm")


In [63]:
# images, labels = load_dataset(10000, val_loader)


In [28]:
# images = images.to(device)
# labels = labels.to(device)
# model_transfer = model_transfer.to(device)

In [ ]:
print("PGD")
pgd_attack_transfer = generate_adv(model_transfer, "pgd")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

pgd_acc = test_attack(val_loader, pgd_attack_transfer, model_transfer, "cuda")

In [ ]:
print("fgsm")
fgsm_attack_transfer = generate_adv(model_transfer, "fgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

fgsm_acc = test_attack(val_loader, fgsm_attack_transfer, model_transfer, "cuda")

In [ ]:
print("nifgsm")
nifgsm_attack_transfer = generate_adv(model_transfer, "nifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

nifgsm_acc = test_attack(val_loader, nifgsm_attack_transfer, model_transfer, "cuda")

In [ ]:
print("vmifgsm")
vmifgsm_attack_transfer = generate_adv(model_transfer, "vmifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

vmifgsm_acc = test_attack(val_loader, vmifgsm_attack_transfer, model_transfer, "cuda")

In [64]:
# print("PGD")
# pgd_attack_transfer = generate_adv(model_transfer, "pgd")
# # pgd_attack_transfer = pgd_attack_transfer.cpu()
# adv_images_pgd_adv_t = pgd_attack_transfer(images, labels)
# acc = clean_accuracy(model_transfer, adv_images_pgd_adv_t, labels)
# print('[Model loaded]')
# print('Acc: %2.2f %%'%(acc*100))
# print("adv model")

PGD


RuntimeError: CUDA out of memory. Tried to allocate 5.61 GiB (GPU 0; 14.56 GiB total capacity; 5.82 GiB already allocated; 709.50 MiB free; 5.84 GiB reserved in total by PyTorch)

In [ ]:
# print("fgsm")
# adv_images_fgsm_adv_t = fgsm_attack_transfer(images, labels)
# acc = clean_accuracy(model_transfer, adv_images_fgsm_adv_t, labels)
# print('[Model loaded]')
# print('Acc: %2.2f %%'%(acc*100))
# print("adv model")

In [ ]:
#  

In [ ]:
# #vmifgsm_attack = generate_adv(net, "vmifgsm")
# print("vmifgsm")
# adv_images_vmifgsm_adv_t =vmifgsm_attack_transfer(images, labels)
# acc = clean_accuracy(model_transfer, adv_images_vmifgsm_adv_t, labels)
# print('[Model loaded]')
# print('Acc: %2.2f %%'%(acc*100))
# print("adv model")

# Adversarial Training 

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import copy
net_adv = copy.deepcopy(model_transfer) #am I doing correct? 
for param in net_adv.parameters():
    print(param)

In [ ]:
params_to_update = net_adv.parameters()

print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in net_adv.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_transfer.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_adv = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()

PATH = './cifar_trans_resnet_adv_pgd.pth'
device = "cuda"
writer = SummaryWriter("Adv_trans_resnet_PGD")
pgd_attack = generate_adv(net_adv, "pgd")

In [ ]:
train(net_adv, train_loader, train_criterion, val_loader, val_criterion, 
              optimizer_adv, 100, writer, 
                 PATH, True, pgd_attack, device="cuda",
                  n_steps_show=1000)

# TODO Compared with from scratch standard training and attacks 

In [ ]:
model_name = "resnet"
num_classes = 10
feature_extract = False

model_scratch, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=False)
train_dataset, val_dataset, train_loader, val_loader = CIFAR10(128, finetune = True, 
                                                               input_size = input_size, test_batch_size=128)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
device = "cuda"

In [ ]:
model_scratch = model_scratch.to(device)
params_to_update = model_scratch.parameters()

print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_scratch.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_scratch.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_scratch = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

# TODO Compared with from scratch adversarial training and attacks 